In [1]:
# Add imports
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
# S3 bucket details
access_key = os.getenv('AWS-ACCESS-KEY')
secret_key = os.getenv('AWS-SECRET-KEY')
bucket_name = os.getenv('AWS-BUCKET-NAME')
json_file_path = "properties.json"
delta_table_path = "properties.parquet"

In [3]:
# Set up Spark session
from pyspark.sql import SparkSession
from delta import *

builder = SparkSession.builder \
    .appName("spark test") \
    .config("spark.hadoop.fs.s3a.access.key", access_key) \
    .config("spark.hadoop.fs.s3a.secret.key", secret_key) \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [4]:
# Read JSON file into dataframe
df = spark.read.option("multiline","true").json(f"s3a://{bucket_name}/{json_file_path}")

In [6]:
# Convert dataframe to Delta Lake table
df.write.format("delta").save(f"s3a://{bucket_name}/{delta_table_path}")

AnalysisException: Found invalid character(s) among ' ,;{}()\n\t=' in the column names of your schema. 
Please enable Column Mapping on your Delta table with mapping mode 'name'.
You can use one of the following commands.

If your table is already on the required protocol version:
ALTER TABLE table_name SET TBLPROPERTIES ('delta.columnMapping.mode' = 'name')

If your table is not on the required protocol version and requires a protocol upgrade:
ALTER TABLE table_name SET TBLPROPERTIES (
   'delta.columnMapping.mode' = 'name',
   'delta.minReaderVersion' = '2',
   'delta.minWriterVersion' = '5')


In [7]:
df.printSchema()

root
 |-- address: struct (nullable = true)
 |    |-- locality: string (nullable = true)
 |    |-- location: struct (nullable = true)
 |    |    |-- latitude: double (nullable = true)
 |    |    |-- longitude: double (nullable = true)
 |    |-- postcode: string (nullable = true)
 |    |-- showAddress: boolean (nullable = true)
 |    |-- state: string (nullable = true)
 |    |-- streetAddress: string (nullable = true)
 |    |-- subdivisionCode: string (nullable = true)
 |    |-- suburb: string (nullable = true)
 |-- advertising: struct (nullable = true)
 |    |-- priceRange: string (nullable = true)
 |    |-- region: string (nullable = true)
 |-- agency: struct (nullable = true)
 |    |-- address: struct (nullable = true)
 |    |    |-- postcode: string (nullable = true)
 |    |    |-- state: string (nullable = true)
 |    |    |-- streetAddress: string (nullable = true)
 |    |    |-- suburb: string (nullable = true)
 |    |-- agencyId: string (nullable = true)
 |    |-- brandingColors

In [8]:
df.show()

+--------------------+--------------------+--------------------+--------------------+-----------+--------------+--------------------+-------+--------------+--------------------+--------------------+--------+-----------+--------------------+--------------------+----------------------+------------+-------------+-------------+--------------------+--------------------+---------+--------------------+-------+------------+--------------------+--------------------+------------+--------------------+------------+--------------+---------+----------------+--------+------------------+--------------------+
|             address|         advertising|              agency|     agencyListingId|applyOnline|          bond|          calculator|channel|classicProject|       dateAvailable|         description|featured|   features|     generalFeatures|              images|inspectionsAndAuctions|isBuyChannel|isRentChannel|isSoldChannel|              lister|             listers|listingId|           mainImage|mi

In [9]:
spark.stop()